In [1]:
import pandas as pd
import os
from tqdm.autonotebook import tqdm

/usr/local/lib/python3.5/dist-packages/tqdm/autonotebook/__init__.py:14: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  " (e.g. in jupyter console)", TqdmExperimentalWarning)


In [2]:
df = pd.read_json("/home/jupyter/datasets/outputevents/tokenized/outputevents.json", orient = "records", dtype="object")

In [3]:
df.head()

,HADM_ID,ICUSTAY_ID,STORETIME,SUBJECT_ID,outputevents
0,177991,225765,2142-09-08 12:08:00,21219,"[Urine, Out, Foley, 40055-200-ml]"
1,177991,225765,2142-09-08 12:08:00,21219,"[Urine, Out, Foley, 40055-200-ml]"
2,177991,225765,2142-09-08 13:39:00,21219,"[Urine, Out, Foley, 40055-120-ml]"
3,177991,225765,2142-09-08 16:17:00,21219,"[Urine, Out, Foley, 40055-100-ml]"
4,177991,225765,2142-09-08 16:17:00,21219,"[Urine, Out, Foley, 40055-200-ml]"


In [4]:
df.info(verbose = True, null_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4349218 entries, 0 to 4349217
Data columns (total 5 columns):
HADM_ID         4349218 non-null object
ICUSTAY_ID      4349218 non-null object
STORETIME       4349218 non-null object
SUBJECT_ID      4349218 non-null object
outputevents    4349218 non-null object
dtypes: object(5)
memory usage: 165.9+ MB


In [5]:
df_ICU = pd.read_csv("~/datasets/raw/ICUSTAYS.csv", dtype = "object")

In [6]:
df_ICU.head()

,ROW_ID,SUBJECT_ID,HADM_ID,ICUSTAY_ID,DBSOURCE,FIRST_CAREUNIT,LAST_CAREUNIT,FIRST_WARDID,LAST_WARDID,INTIME,OUTTIME,LOS
0,365,268,110404,280836,carevue,MICU,MICU,52,52,2198-02-14 23:27:38,2198-02-18 05:26:11,3.249
1,366,269,106296,206613,carevue,MICU,MICU,52,52,2170-11-05 11:05:29,2170-11-08 17:46:57,3.2788
2,367,270,188028,220345,carevue,CCU,CCU,57,57,2128-06-24 15:05:20,2128-06-27 12:32:29,2.8939
3,368,271,173727,249196,carevue,MICU,SICU,52,23,2120-08-07 23:12:42,2120-08-10 00:39:04,2.06
4,369,272,164716,210407,carevue,CCU,CCU,57,57,2186-12-25 21:08:04,2186-12-27 12:01:13,1.6202


In [7]:
df_ICU.info(verbose = True, null_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 61532 entries, 0 to 61531
Data columns (total 12 columns):
ROW_ID            61532 non-null object
SUBJECT_ID        61532 non-null object
HADM_ID           61532 non-null object
ICUSTAY_ID        61532 non-null object
DBSOURCE          61532 non-null object
FIRST_CAREUNIT    61532 non-null object
LAST_CAREUNIT     61532 non-null object
FIRST_WARDID      61532 non-null object
LAST_WARDID       61532 non-null object
INTIME            61532 non-null object
OUTTIME           61522 non-null object
LOS               61522 non-null object
dtypes: object(12)
memory usage: 5.6+ MB


In [8]:
# Selecting list of HADM_ID with single ICUSTAY
df_single_ICU = df_ICU.groupby(by=['HADM_ID'], as_index=False).agg({'ICUSTAY_ID': 'count'})
df_single_ICU  = df_single_ICU[df_single_ICU['ICUSTAY_ID']==1]['HADM_ID']

In [9]:
#number of HADM_ID with single ICUSTAY
len(df_single_ICU)

54526

In [10]:
# Dropping HADM_ID with mutiple ICUSTAYS
df = df[df['HADM_ID'].isin(df_single_ICU)].reset_index(drop = True)

In [11]:
df.info(verbose = True, null_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3731510 entries, 0 to 3731509
Data columns (total 5 columns):
HADM_ID         3731510 non-null object
ICUSTAY_ID      3731510 non-null object
STORETIME       3731510 non-null object
SUBJECT_ID      3731510 non-null object
outputevents    3731510 non-null object
dtypes: object(5)
memory usage: 142.3+ MB


In [12]:
df = df.merge(df_ICU[['ICUSTAY_ID','INTIME']], how='left',left_on='ICUSTAY_ID', right_on='ICUSTAY_ID')

In [13]:
df = df.rename(columns={'INTIME':'ICU_INTIME'})

In [14]:
df.head()

,HADM_ID,ICUSTAY_ID,STORETIME,SUBJECT_ID,outputevents,ICU_INTIME
0,177991,225765,2142-09-08 12:08:00,21219,"[Urine, Out, Foley, 40055-200-ml]",2142-08-30 16:34:00
1,177991,225765,2142-09-08 12:08:00,21219,"[Urine, Out, Foley, 40055-200-ml]",2142-08-30 16:34:00
2,177991,225765,2142-09-08 13:39:00,21219,"[Urine, Out, Foley, 40055-120-ml]",2142-08-30 16:34:00
3,177991,225765,2142-09-08 16:17:00,21219,"[Urine, Out, Foley, 40055-100-ml]",2142-08-30 16:34:00
4,177991,225765,2142-09-08 16:17:00,21219,"[Urine, Out, Foley, 40055-200-ml]",2142-08-30 16:34:00


In [15]:
df = df.fillna('NaN')

In [16]:
#creating a list HADM_ID of records with missing ICUSTAYID
missing_ICUSTAYID_index = df[(df['HADM_ID']!='NaN') & (df['ICUSTAY_ID']=='NaN')]['HADM_ID'].unique()

In [17]:
missing_ICUSTAYID_index

array(['184129', '126274', '138066', '124738', '169182', '148324',
       '102005', '190691', '146232', '188201', '127498', '130232',
       '171006', '127370', '186210', '127928', '110661', '199274',
       '189055', '159445', '123802', '118313', '111526', '164955',
       '171549', '161499', '155705', '181572', '170464', '172591',
       '168108', '159300', '112341', '108944', '141085', '116826',
       '186658', '170946', '163189', '114105', '147876', '199898',
       '143664', '180173', '161858', '107913', '133955', '192997',
       '125603', '116787', '179134', '136508', '153483', '152694',
       '170286', '157046', '180715', '147345', '192002', '103793',
       '145083', '174140', '114498', '185976', '101748', '100855',
       '156870', '137169', '171946', '147191', '187154', '170894',
       '150980', '178021', '169331', '112095', '127741', '197889',
       '129846', '135258', '160956', '113043', '190225', '102535',
       '171113', '104872', '173113', '120164', '144589', '1632

In [18]:
%%time
# Replace records with no ICUSTAYID with ICU INTIME associated with HADM_ID
for i in tqdm(missing_ICUSTAYID_index):
    x = df_ICU[df_ICU['HADM_ID']==i]['INTIME'].to_list()[0]
    df.loc[(df['HADM_ID']==i) & (df['ICUSTAY_ID']=='NaN'),'ICU_INTIME'] = x


CPU times: user 3min 38s, sys: 17.9 s, total: 3min 56s
Wall time: 3min 56s


In [19]:
df['STORETIME'] = pd.to_datetime(df['STORETIME'], errors='coerce')
df['ICU_INTIME'] = pd.to_datetime(df['ICU_INTIME'], errors='coerce')

In [20]:
%%time
df['icu_tdelta'] = df['STORETIME'] - df['ICU_INTIME']
df['icu_tdelta'] = df['icu_tdelta'].map(lambda x: x.total_seconds())

CPU times: user 31.4 s, sys: 760 ms, total: 32.2 s
Wall time: 32.2 s


In [21]:
df['STORETIME'] = df['STORETIME'].map(lambda x: str(x))

In [22]:
df.head()

,HADM_ID,ICUSTAY_ID,STORETIME,SUBJECT_ID,outputevents,ICU_INTIME,icu_tdelta
0,177991,225765,2142-09-08 12:08:00,21219,"[Urine, Out, Foley, 40055-200-ml]",2142-08-30 16:34:00,761640.0
1,177991,225765,2142-09-08 12:08:00,21219,"[Urine, Out, Foley, 40055-200-ml]",2142-08-30 16:34:00,761640.0
2,177991,225765,2142-09-08 13:39:00,21219,"[Urine, Out, Foley, 40055-120-ml]",2142-08-30 16:34:00,767100.0
3,177991,225765,2142-09-08 16:17:00,21219,"[Urine, Out, Foley, 40055-100-ml]",2142-08-30 16:34:00,776580.0
4,177991,225765,2142-09-08 16:17:00,21219,"[Urine, Out, Foley, 40055-200-ml]",2142-08-30 16:34:00,776580.0


In [23]:
sec_24hrs = 24*60*60 #total seconds in 24 hrs
sec_48hrs = 48*60*60 #total seconds in 48 hrs

In [24]:
print("Total Records: " + str(len(df)))
print("Total Records Prior to 24hrs in ICU: " +str(len(df[df['icu_tdelta']<sec_24hrs])))
print("Total Records Prior to 48hrs in ICU: " +str(len(df[df['icu_tdelta']<sec_48hrs])))

Total Records: 3731510
Total Records Prior to 24hrs in ICU: 918469
Total Records Prior to 48hrs in ICU: 1530961


In [25]:
# create output path
mypath_output = "/home/jupyter/datasets/data_before_24hrs_icu/"
import os
os.makedirs(mypath_output, exist_ok=True)

In [26]:
# extract data recorded in ans prior to first 24hrs of ICU stay
df[df['icu_tdelta']<sec_24hrs].to_json(mypath_output+"outputevents.json", orient = 'records')

In [27]:
# create output path
mypath_output = "/home/jupyter/datasets/data_before_48hrs_icu/"
import os
os.makedirs(mypath_output, exist_ok=True)

In [28]:
# extract data recorded in ans prior to first 48hrs of ICU stay
df[df['icu_tdelta']<sec_48hrs].to_json(mypath_output+"outputevents.json", orient = 'records')